# Eco-Bici CDMX

## 1. Extract & Load of data

### 1.1: Preparing environment

Installing libraries: pandas, requests beautifulsoup4 

In [5]:
!pip install requests pandas beautifulsoup4

### 1.2: Exploring EcoBici Web

In [7]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
# define URL page that contains the data 
url_main_page = "https://ecobici.cdmx.gob.mx/datos-abiertos/"

In [9]:
# using request to get information
print("Exploring the page")
page = requests.get(url_main_page)
print("Page obtained")

Exploring the page
Page obtained


In [10]:
# HTML lecture
soup = BeautifulSoup(page.content,"html.parser")

In [11]:
# looking and finding <a> labels
all_links = soup.find_all('a')

for links in (all_links):
    print(f"Found {links} <a> labels")

print(f"\nTotal <a> labels found: {len(all_links)}")

Found <a href="/">
<img alt="Ecobici" class="c-header__logo-img c-header__logo-img--desktop" src="/wp-content/uploads/2022/06/0ecobici-gob_logo.png"/>
<img alt="Ecobici" class="c-header__logo-img c-header__logo-img--mobile" src="/wp-content/uploads/2022/06/0ecobici-gob_logo.png"/>
</a> <a> labels
Found <a class="c-button c-button--small c-header_mobile-book-now" href="https://usuario.ecobici.cdmx.gob.mx/">Regístrate</a> <a> labels
Found <a>Ecobici</a> <a> labels
Found <a href="/conoce-sistema/">Conoce Ecobici</a> <a> labels
Found <a aria-current="page" href="/datos-abiertos/">Datos Abiertos</a> <a> labels
Found <a href="/encuestas-y-estudios-de-uso/">Encuestas y estudios</a> <a> labels
Found <a href="/informacion-ciclista/">Información ciclista</a> <a> labels
Found <a href="/estadisticas/">Estadísticas</a> <a> labels
Found <a>▾</a> <a> labels
Found <a href="/planes-y-tarifas/">Planes y tarifas</a> <a> labels
Found <a href="/mapa/">Mapa</a> <a> labels
Found <a href="/conoce-la-app/">Con

In [12]:
# Filtering <a> labels .csv

links_csvs = []

base_url = "https://ecobici.cdmx.gob.mx"

for csvs in all_links:
    csv_href = csvs.get('href')
    if csv_href:
        if ".csv" in csv_href:
            links_csvs.append(base_url+csv_href)

print("-- Links found --")
print(f"Csvs found: {len(links_csvs)}")

# slicing list
links_csvs_limit = links_csvs[7:10]

print(f"\nTotal csvs used: {len(links_csvs_limit)}")
print(links_csvs_limit)

-- Links found --
Csvs found: 189

Total csvs used: 3
['https://ecobici.cdmx.gob.mx/wp-content/uploads/2025/09/2025-08.csv', 'https://ecobici.cdmx.gob.mx/wp-content/uploads/2025/10/2025-09.csv', 'https://ecobici.cdmx.gob.mx/wp-content/uploads/2025/11/2025-10.csv']


### 1.3: Using Pandas for download and concat files

In [14]:
# save dataframe we'll use
df_list = []
print(f'Start downloading and lecture: {len(links_csvs_limit)} files')

for url in links_csvs_limit:
    print(f"Processing: {url.split('/')[-1]}")
    df_temporal = pd.read_csv(url)
    df_list.append(df_temporal)
    
print("All files were processed")

Start downloading and lecture: 3 files
Processing: 2025-08.csv
Processing: 2025-09.csv
Processing: 2025-10.csv
All files were processed


In [15]:
# Merge files 
print(f"Merging all files in one single Dataframe Master")
df_master = pd.concat(df_list)
print("--- ¡Completed! ---")

Merging all files in one single Dataframe Master
--- ¡Completed! ---


In [16]:
print(f"Dimensions dataframe: {df_master.shape}")

Dimensions dataframe: (4951828, 9)


In [17]:
df_master.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha_Arribo,Hora_Arribo
0,F,42.0,6401327,107-108,31/07/2025,23:51:05,104,01/08/2025,00:00:07
1,M,27.0,8063216,140,31/07/2025,23:54:46,158-159,01/08/2025,00:00:13
2,F,22.0,3710100,141,31/07/2025,23:55:49,158-159,01/08/2025,00:00:29
3,M,24.0,6243661,548,31/07/2025,23:48:39,482,01/08/2025,00:00:29
4,M,25.0,4258406,567,31/07/2025,23:40:31,567,01/08/2025,00:00:33


In [18]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4951828 entries, 0 to 1754352
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Genero_Usuario         object 
 1   Edad_Usuario           float64
 2   Bici                   int64  
 3   Ciclo_Estacion_Retiro  object 
 4   Fecha_Retiro           object 
 5   Hora_Retiro            object 
 6   Ciclo_EstacionArribo   object 
 7   Fecha_Arribo           object 
 8   Hora_Arribo            object 
dtypes: float64(1), int64(1), object(7)
memory usage: 377.8+ MB


### 1.5: Transforming

We will use an time_difference KPI, we need to modify/convert columns dates

In [21]:
# Create new column Timestamp for Leaving-Retiro and Arriving- Arribo
df_master['Timestamp_Retiro'] = df_master['Fecha_Retiro'] + ' ' + df_master['Hora_Retiro']

df_master['Timestamp_Arribo'] = df_master['Fecha_Arribo'] + ' ' + df_master['Hora_Arribo']

In [22]:
# Convert datatype to timestamp date
# Set format
date_format = '%d/%m/%Y %H:%M:%S'

In [23]:
df_master['Timestamp_Retiro'] = pd.to_datetime(df_master['Timestamp_Retiro'], format=date_format)

df_master['Timestamp_Arribo'] = pd.to_datetime(df_master['Timestamp_Arribo'], format=date_format)

print('Dates transformed')

Dates transformed


In [24]:
# creating metric- time difference 
diferencia_tiempo = df_master['Timestamp_Arribo'] - df_master['Timestamp_Retiro']

In [25]:
# convert to minutes
# create new columne time difference
df_master['Duracion_Minutos'] = diferencia_tiempo.dt.total_seconds() / 60

In [26]:
df_master[['Timestamp_Retiro', 'Timestamp_Arribo', 'Duracion_Minutos']].head()

,Timestamp_Retiro,Timestamp_Arribo,Duracion_Minutos
0,2025-07-31 23:51:05,2025-08-01 00:00:07,9.033333
1,2025-07-31 23:54:46,2025-08-01 00:00:13,5.450000
2,2025-07-31 23:55:49,2025-08-01 00:00:29,4.666667
3,2025-07-31 23:48:39,2025-08-01 00:00:29,11.833333
4,2025-07-31 23:40:31,2025-08-01 00:00:33,20.033333


In [27]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4951828 entries, 0 to 1754352
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Genero_Usuario         object        
 1   Edad_Usuario           float64       
 2   Bici                   int64         
 3   Ciclo_Estacion_Retiro  object        
 4   Fecha_Retiro           object        
 5   Hora_Retiro            object        
 6   Ciclo_EstacionArribo   object        
 7   Fecha_Arribo           object        
 8   Hora_Arribo            object        
 9   Timestamp_Retiro       datetime64[ns]
 10  Timestamp_Arribo       datetime64[ns]
 11  Duracion_Minutos       float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(7)
memory usage: 491.1+ MB


In [59]:
df_master['Genero_Usuario'].unique()

array(['F', 'M', 'O', '?', nan], dtype=object)

In [65]:
df_master['Edad_Usuario'].isna().sum()

223

In [69]:
df_master['Bici'].isna().sum()

0

In [83]:
df_master['Ciclo_Estacion_Retiro'].unique()

array(['107-108', '140', '141', '548', '567', '044', '074', '013', '137',
       '237-238', '491', '397', '428', '068', '449', '389', '087', '031',
       '072', '480', '178', '073', '064', '082', '447', '192-193', '259',
       '040', '119', '216', '131', '270', '123', '133', '675', '170',
       '205', '045', '151', '144', '057', '271-272', '504', '034', '595',
       '256', '118', '091', '458', '651', '223', '242', '496', '624',
       '055', '221', '260', '462', '128', '603', '402', '029', '041',
       '047', '606', '708', '025', '048', '481', '545', '494', '027',
       '381', '042', '061', '517', '373', '386', '277', '563', '551',
       '513', '121', '208', '028', '209', '455', '250', '372', '257',
       '217', '464', '083', '134', '030', '502', '075', '546', '166',
       '398', '399', '495', '361', '009', '529', '049', '584', '135',
       '336', '008', '230', '476', '032', '553', '244', '359', '444',
       '401', '578', '126', '598', '052', '568', '006', '069', '054',
    

In [97]:
df_master['Ciclo_EstacionArribo'].unique()

array(['104', '158-159', '482', '567', '111', '205', '563', '035', '160',
       '405', '430', '032', '062', '634', '573', '262', '538', '271-272',
       '466', '016', '083', '167', '612', '001', '571', '002', '109',
       '256', '595', '594', '172', '217', '141', '532', '182', '145',
       '034', '685', '287', '169', '613', '579', '545', '152', '485',
       '448', '181', '591', '291', '260', '031', '617', '221', '506',
       '497', '636', '130', '073', '270', '183', '540', '051', '489',
       '620', '029', '074', '065', '146', '144', '602', '708', '577',
       '565', '091', '011', '488', '209', '086', '553', '534', '263',
       '356', '580', '522', '175', '310', '370', '106', '064', '025',
       '510', '244', '464', '092', '470', '237-238', '107-108', '505',
       '286', '013', '298', '325', '015', '047', '028', '551', '179',
       '413', '407', '501', '066', '110', '155', '008', '219', '071',
       '645', '502', '544', '555', '481', '372', '068', '061', '409',
       '578

In [93]:
df_master[(df_master['Bici'] == 8205254) & (df_master['Edad_Usuario']==50)]

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha_Arribo,Hora_Arribo,Timestamp_Retiro,Timestamp_Arribo,Duracion_Minutos
102765,M,50.0,8205254,545,03/08/2025,13:49:44,544,03/08/2025,13:51:52,2025-08-03 13:49:44,2025-08-03 13:51:52,2.133333
1293332,M,50.0,8205254,170,26/08/2025,09:31:24,473,26/08/2025,09:59:36,2025-08-26 09:31:24,2025-08-26 09:59:36,28.200000
682661,M,50.0,8205254,641,13/09/2025,10:13:16,639,13/09/2025,10:17:33,2025-09-13 10:13:16,2025-09-13 10:17:33,4.283333
1001666,M,50.0,8205254,189,19/09/2025,23:54:14,152,20/09/2025,00:06:49,2025-09-19 23:54:14,2025-09-20 00:06:49,12.583333
1257401,M,50.0,8205254,340,24/09/2025,19:01:44,277,24/09/2025,19:12:26,2025-09-24 19:01:44,2025-09-24 19:12:26,10.700000
1490812,M,50.0,8205254,089,29/09/2025,16:15:58,559,29/09/2025,16:29:10,2025-09-29 16:15:58,2025-09-29 16:29:10,13.200000
1492805,M,50.0,8205254,559,29/09/2025,16:31:17,630,29/09/2025,16:57:21,2025-09-29 16:31:17,2025-09-29 16:57:21,26.066667
1502048,M,50.0,8205254,627,29/09/2025,18:34:49,616,29/09/2025,18:38:07,2025-09-29 18:34:49,2025-09-29 18:38:07,3.300000
887604,M,50.0,8205254,293,17/10/2025,09:52:02,Temporal - 2da Sección Bosque Chapultepec,17/10/2025,09:56:48,2025-10-17 09:52:02,2025-10-17 09:56:48,4.766667
1497657,M,50.0,8205254,292,28/10/2025,07:40:28,360,28/10/2025,08:08:18,2025-10-28 07:40:28,2025-10-28 08:08:18,27.833333


## Connect to MySQL Workbench

### 2.1: Installing PyMySQL

In [51]:
!pip install PyMySQL

### 2.2: Create Database in MySQL

### 2.3: Connection MySQL

In [57]:
import sqlalchemy

usuario = 'root'
contrasena = 'abcd1234'
host = '127.0.0.1'
basedatos = 'ecobici_db'

print("Creating connection to MySQL")
engine = sqlalchemy.create_engine(f"mysql+pymysql://{usuario}:{contrasena}@{host}/{basedatos}")

print(f"Loading {df_master.shape[0]} rows to Database")

df_master.to_sql(
    name='viajes',
    con=engine,
    if_exists='replace',
    index=False,
    chunksize=10000   
)

print("Loaded Successful, Data stored in ecobici_db ")

Creating connection to MySQL
Loading 4951828 rows to Database
Loaded Successful, Data stored in ecobici_db 
